In [ ]:
pip install google-cloud-videointelligence

In [1]:
import io
from PIL import Image, ImageDraw, ImageFont
from google.cloud import videointelligence
import cv2


def detect_logo(local_file_path):
    """Performs asynchronous video annotation for logo recognition on a local file."""

    client = videointelligence.VideoIntelligenceServiceClient()

    with io.open(local_file_path, "rb") as f:
        input_content = f.read()
    features = [videointelligence.enums.Feature.LOGO_RECOGNITION]

    operation = client.annotate_video(input_content=input_content, features=features)

    print(u"Waiting for operation to complete...")
    response = operation.result()
#     print_detected_logos(response)
    # Get the first response, since we sent only one video.
    annotation_result = response.annotation_results[0]

    # Annotations for list of logos detected, tracked and recognized in video.
    for logo_recognition_annotation in annotation_result.logo_recognition_annotations:
        entity = logo_recognition_annotation.entity
#         print_logo_frames(response, entity.entity_id)
        # Opaque entity ID. Some IDs may be available in [Google Knowledge Graph
        # Search API](https://developers.google.com/knowledge-graph/).
        print(u"Entity Id : {}".format(entity.entity_id))

        print(u"Description : {}".format(entity.description))

        # All logo tracks where the recognized logo appears. Each track corresponds
        # to one logo instance appearing in consecutive frames.
        for track in logo_recognition_annotation.tracks:
            # Video segment of a track.
            print(
                u"\n\tStart Time Offset : {}.{}".format(
                    track.segment.start_time_offset.seconds,
                    track.segment.start_time_offset.nanos,
                )
            )
            print(
                u"\tEnd Time Offset : {}.{}".format(
                    track.segment.end_time_offset.seconds,
                    track.segment.end_time_offset.nanos,
                )
            )
            print(u"\tConfidence : {}".format(track.confidence))

            # The object with timestamp and attributes per frame in the track.
            for timestamped_object in track.timestamped_objects:

                # Normalized Bounding box in a frame, where the object is located.
                normalized_bounding_box = timestamped_object.normalized_bounding_box
                print(u"\n\t\tLeft : {}".format(normalized_bounding_box.left))
                print(u"\t\tTop : {}".format(normalized_bounding_box.top))
                print(u"\t\tRight : {}".format(normalized_bounding_box.right))
                print(u"\t\tBottom : {}".format(normalized_bounding_box.bottom))

                # Optional. The attributes of the object in the bounding box.
                for attribute in timestamped_object.attributes:
                    print(u"\n\t\t\tName : {}".format(attribute.name))
                    print(u"\t\t\tConfidence : {}".format(attribute.confidence))
                    print(u"\t\t\tValue : {}".format(attribute.value))

            # Optional. Attributes in the track level.
            for track_attribute in track.attributes:
                print(u"\n\t\tName : {}".format(track_attribute.name))
                print(u"\t\tConfidence : {}".format(track_attribute.confidence))
                print(u"\t\tValue : {}".format(track_attribute.value))

        # All video segments where the recognized logo appears. There might be
        # multiple instances of the same logo class appearing in one VideoSegment.
        for segment in logo_recognition_annotation.segments:
            print(
                u"\n\tStart Time Offset : {}.{}".format(
                    segment.start_time_offset.seconds, segment.start_time_offset.nanos,
                )
            )
            print(
                u"\tEnd Time Offset : {}.{}".format(
                    segment.end_time_offset.seconds, segment.end_time_offset.nanos,
                )
            )

def print_detected_logos(response):
    # First result only, as a single video is processed
    annotations = response.annotation_results[0].logo_recognition_annotations

    print(f' Detected logos: {len(annotations)} '.center(80, '-'))
    for annotation in annotations:
        entity = annotation.entity
        entity_id = entity.entity_id
        description = entity.description
        for track in annotation.tracks:
            confidence = track.confidence
            start_ms = track.segment.start_time_offset.ToMilliseconds()
            end_ms = track.segment.end_time_offset.ToMilliseconds()
            logo_frames = len(track.timestamped_objects)
            print(f'{confidence:4.0%}',
                  f'{start_ms:>7,}',
                  f'{end_ms:>7,}',
                  f'{logo_frames:>3} fr.',
                  f'{entity_id:<15}',
                  f'{description}',
                  sep=' | ')
            
def print_logo_frames(response, entity_id):
    def keep_annotation(annotation):
        return annotation.entity.entity_id == entity_id

    # First result only, as a single video is processed
    annotations = response.annotation_results[0].logo_recognition_annotations
    annotations = [a for a in annotations if keep_annotation(a)]
    for annotation in annotations:
        description = annotation.entity.description
        for track in annotation.tracks:
            confidence = track.confidence
            print(f' {description},'
                  f' confidence: {confidence:.0%},'
                  f' frames: {len(track.timestamped_objects)} '
                  .center(80, '-'))
            for timestamped_object in track.timestamped_objects:
                frame_ms = timestamped_object.time_offset.ToMilliseconds()
                box = timestamped_object.normalized_bounding_box
                print(f'{frame_ms:>7,}',
                      f'({box.left:.5f}, {box.top:.5f})',
                      f'({box.right:.5f}, {box.bottom:.5f})',
                      sep=' | ')
                
detect_logo('nike_movie2.mp4')

NameError: name 'io' is not defined

In [3]:
def play_video():
    capture = cv2.VideoCapture("nike_movie2.mp4")
    while True:
        if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
            capture.open("nike_movie2.mp4")

        ret, frame = capture.read()
        cv2.imshow("VideoFrame", frame)

        if cv2.waitKey(33) > 0: break

    capture.release()
    cv2.destroyAllWindows()

In [4]:
import cv2

CAM_ID = 0
def capture(camid = CAM_ID):
    cam = cv2.VideoCapture(camid)
    if cam.isOpened() == False:
        print ('cant open the cam (%d)' % camid)
        return None

    ret, frame = cam.read()
    if frame is None:
        print ('frame is not exist')
        return None
    
    # png로 압축 없이 영상 저장 
    cv2.imwrite('messigray.png',frame, params=[cv2.IMWRITE_PNG_COMPRESSION,0])
    cam.release()